In [1]:
import ROOT as R
from os import listdir
from os.path import isfile, join
import sys
sys.path.insert(1,"../plotting/")
from samples import configure_samples
R.EnableImplicitMT()
%jsroot on

Welcome to JupyROOT 6.24/04


In [2]:
d_samp, d_type, d_reg = configure_samples()

In [3]:
mypath = "/storage/eirikgr/ANAntuples/PHYS_Bkgs_mc16e/"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
df = {}
for of in onlyfiles:
    if not of.endswith("_merged_processed.root"): continue
    typ = of.replace("_merged_processed.root","")
    print(typ)
    df[typ] = R.RDataFrame("%s_NoSys"%typ,mypath+"/"+of)

PHWW
PHWZ
PHZZ
Vgamma
Wjets_extension
Wjets
Zjets_extension
Zjets
diboson
higgs
lowMassDY
singleTop
topOther
triboson
ttbar


In [4]:
mypath = "/scratch/eirikgr/ANAoutput/Wed_Sep_01_2021_02L_NTUP_data18_TEST"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
for of in onlyfiles:
    if not of.endswith("_merged_processed_COPY_0_0.root"): continue
    treename = of.replace("_merged_processed_COPY_0_0.root","")
    typ = treename
    print(typ)
    df[typ] = R.RDataFrame(treename,mypath+"/"+of)

data18


In [5]:
#mypath = "/scratch/eirikgr/ANAoutput/Tue_Aug_31_2021_02L_NTUP_data18_TEST/"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
for of in onlyfiles:
    if not of.endswith("_merged_processed_COPY_0_0.root"): continue
    typ = "FNP"
    treename = of.replace("_merged_processed_COPY_0_0.root","")
    print(typ)
    df[typ] = R.RDataFrame(treename,mypath+"/"+of)

FNP


In [6]:
#R.gROOT.ProcessLine(".L functions.cxx+");
R.gSystem.AddDynamicPath("-I/home/eirikgr/myNtupAnalysis/jupyter-notebooks/")
R.gInterpreter.Declare('#include "Cfunctions.h"') # Header with the definition of the myFilter function
R.gSystem.Load("Cfunctions.so") # Library with the myFilter function

-2

Error in <TUnixSystem::Load>: version mismatch, /home/eirikgr/myNtupAnalysis/jupyter-notebooks/Cfunctions.so = 62402, ROOT = 62404


In [7]:
cols = df["PHWW"].GetColumnNames()
variables = []
blacklist = ["lepIFFClass", "lepEgMotherPdgId"]
plotdic = {}
for c in cols:
    typ = df["PHWW"].GetColumnType(c)
    if "bool" in typ or "Bool_t" in typ: continue
    #print(typ)
    cstr = str(c)
    if cstr in blacklist: continue
    if "lep" in cstr and not ("HLT" in cstr or "Weight" in cstr or "Iso" in 
                              cstr or "Truth" in cstr or "Origin" in cstr or "Type" in cstr
                             or "SF" in cstr):
        variables.append(c)
        print(c)
        plotdic[c] = {"nbin":0,"max":0,"min":0}

lepFlavor
lepCharge
lepAuthor
lepPt
lepEta
lepPhi
lepM
lepD0
lepD0Sig
lepZ0
lepZ0SinTheta
lepECIDS


In [8]:
for key in plotdic.keys():
    print(key)

lepFlavor
lepCharge
lepAuthor
lepPt
lepEta
lepPhi
lepM
lepD0
lepD0Sig
lepZ0
lepZ0SinTheta
lepECIDS


In [9]:
plotdic["lepFlavor"]["nbin"] = 2
plotdic["lepFlavor"]["nmax"] = 2
plotdic["lepFlavor"]["nmin"] = 0

plotdic["lepCharge"]["nbin"] = 3
plotdic["lepCharge"]["nmax"] = 1
plotdic["lepCharge"]["nmin"] = -1

plotdic["lepAuthor"]["nbin"] = 10
plotdic["lepAuthor"]["nmax"] = 10
plotdic["lepAuthor"]["nmin"] = 0

plotdic["lepPt"]["nbin"] = 100
plotdic["lepPt"]["nmax"] = 1000
plotdic["lepPt"]["nmin"] = 0

plotdic["lepEta"]["nbin"] = 80
plotdic["lepEta"]["nmax"] = 4
plotdic["lepEta"]["nmin"] = -4

plotdic["lepPhi"]["nbin"] = 80
plotdic["lepPhi"]["nmax"] = 4
plotdic["lepPhi"]["nmin"] = -4

plotdic["lepM"]["nbin"] = 100
plotdic["lepM"]["nmax"] = 1000
plotdic["lepM"]["nmin"] = 0

plotdic["lepD0"]["nbin"] = 200
plotdic["lepD0"]["nmax"] = 10
plotdic["lepD0"]["nmin"] = -10

plotdic["lepZ0"]["nbin"] = 200
plotdic["lepZ0"]["nmax"] = 10
plotdic["lepZ0"]["nmin"] = -10

plotdic["lepD0Sig"]["nbin"] = 100
plotdic["lepD0Sig"]["nmax"] = 10
plotdic["lepD0Sig"]["nmin"] = 0

plotdic["lepZ0SinTheta"]["nbin"] = 100
plotdic["lepZ0SinTheta"]["nmax"] = 10
plotdic["lepZ0SinTheta"]["nmin"] = 0

plotdic["lepECIDS"]["nbin"] = 2
plotdic["lepECIDS"]["nmax"] = 2
plotdic["lepECIDS"]["nmin"] = 0

plotdic["mymll"] = {}
plotdic["mymll"]["nbin"] = 200
plotdic["mymll"]["nmax"] = 2000
plotdic["mymll"]["nmin"] = 0

plotdic["mymt2"] = {}
plotdic["mymt2"]["nbin"] = 200
plotdic["mymt2"]["nmax"] = 2000
plotdic["mymt2"]["nmin"] = 0


In [10]:
for key in df.keys():
    if "data" in key:
        df[key] = df[key].Define("wgt","1.0")
    elif "FNP" in key:
        df[key] = df[key].Define("wgt","FNP_WEIGHTS")
    else:
        df[key] = df[key].Define("scalelumi","(RandomRunNumber) < 320000 ? 36207.65 : (((RandomRunNumber) > 320000 && (RandomRunNumber) < 348000) ? 44307.4 : 58450.1);")
        df[key] = df[key].Define("wgt","((genWeight) * (eventWeight) * (leptonWeight) * (jvtWeight) * (bTagWeight) * (pileupWeight) * scalelumi)")
    df[key] = df[key].Define("baseline_el","lepPt > 9.0 && lepFlavor == 1 && abs(lepEta) < 2.47" + \
                             "&& abs(lepEta) < 2.47 && lepLoose && lepPassBL && abs(lepZ0SinTheta) < 0.5" + \
                            "&& lepPassOR")
    df[key] = df[key].Define("baseline_mu","lepPt > 9.0 && lepFlavor == 2 && abs(lepEta) < 2.6 " +\
                             "&& lepMedium && abs(lepZ0SinTheta) < 0.5")
    df[key] = df[key].Define("signal_el","baseline_el && lepD0Sig < 5 && lepTight && lepIsoFCLoose")
    df[key] = df[key].Define("signal_mu","baseline_mu && lepD0Sig < 3 && lepIsoFCLoose")
    df[key] = df[key].Define("baseline_lep","baseline_el || baseline_mu")
    df[key] = df[key].Define("signal_lep","signal_el || signal_mu")
    df[key] = df[key].Define("isOS","isOS(lepCharge[baseline_lep])")
    df[key] = df[key].Filter("Sum(baseline_lep) == 2","2 baseline leptons")
    if not key == "FNP": 
        df[key] = df[key].Filter("Sum(signal_lep) == 2","2 signal leptons")
        df[key] = df[key].Define("mymll","ComputeInvariantMass(lepPt[signal_lep],lepEta[signal_lep],lepPhi[signal_lep],lepM[signal_lep])")    
        df[key] = df[key].Define("mymt2","calcMT2(lepPt[signal_lep],lepEta[signal_lep],lepPhi[signal_lep],lepM[signal_lep], met_Et, met_Phi)")    
    else:
        df[key] = df[key].Define("mymll","ComputeInvariantMass(lepPt[baseline_lep],lepEta[baseline_lep],lepPhi[baseline_lep],lepM[baseline_lep])")    
        df[key] = df[key].Define("mymt2","calcMT2(lepPt[baseline_lep],lepEta[baseline_lep],lepPhi[baseline_lep],lepM[baseline_lep], met_Et, met_Phi)")    
    df[key].Filter("isOS","opposite sign")
    

In [11]:
histo = {}
allhisto = []
variables = ["mymll"]#,"mymt2"]
for v in variables:
    print(v)
    if not v in plotdic.keys():
        print("No information on binning for %s"%v)
        continue
    histo[v] = {}
    for key in df.keys():
        histo[v][key] = df[key].Histo1D(("h_%s_%s"%(v,key),"h_%s_%s"%(v,key),plotdic[v]["nbin"],
                                         plotdic[v]["nmin"],plotdic[v]["nmax"]),v,"wgt")
        allhisto.append(histo[v][key])

mymll


In [12]:
print("Number of histograms = %i"%len(allhisto))
print("Number of runs = %i"%df["FNP"].GetNRuns())
print("Number of slots = %i"%df["FNP"].GetNSlots())
#df["FNP"].Describe()
#%%time
#%%time


Number of histograms = 17
Number of runs = 0
Number of slots = 256


In [13]:
graph = df["FNP"].SaveGraph(1)

AttributeError: 'RInterface<ROOT::Detail::RDF::RJittedFilter,void>' object has no attribute 'SaveGraph'

In [14]:
%%time
R.RDF.RunGraphs(allhisto)

CPU times: user 2h 20min 3s, sys: 52min 17s, total: 3h 12min 20s
Wall time: 13min 49s


In [15]:
stack = {}
data = {}
legend = {}
for v in histo.keys():
    # Add legend
    legend[v] = R.TLegend(0.60, 0.60, 0.8, 0.85)
    legend[v].SetTextFont(42)
    legend[v].SetFillStyle(0)
    legend[v].SetBorderSize(0)
    legend[v].SetTextSize(0.04)
    stack[v] = R.THStack()
    data[v] = {}
    for key in histo[v].keys():
        if not key in d_samp.keys():
            print("skipping %s"%key)
            continue
        if not "data" in key:
            h = histo[v][key].GetValue()
            h.SetFillColor(d_samp[key]['f_color'])
            h.SetLineColor(d_samp[key]['f_color'])
            stack[v].Add(h)
            legend[v].AddEntry(h,"%-s"%key,"f")
        else:
            data[v][key] = histo[v][key].GetValue()
            data[v][key].SetMarkerStyle(d_samp[key]['m_type'])
            data[v][key].SetLineColor(d_samp[key]['f_color'])
            data[v][key].SetMarkerColor(d_samp[key]['f_color'])
            legend[v].AddEntry(data[v][key],"%-s"%key,"lp")

skipping diboson


In [16]:
%%time
datak = "data18"
lumi = 58450.1
sumMC = {}
ratio = {}
v = "mymll"
# Create canvas with pad
c = R.TCanvas("c", "", 900, 700)
c.Draw()
R.gStyle.SetOptStat(0)
pad = R.TPad("upper_pad", "", 0, 0.2, 1, 1.0)
pad2 = R.TPad("lower_pad", "", 0, 0, 1, 0.2)
pad.SetTickx(False)
pad.SetTicky(False)
pad.SetBottomMargin(0.005)
pad.SetLogy()
pad.Draw()
pad2.Draw()
pad.cd()
stack[v].Draw("HIST")

sumMC[v] = stack[v].GetStack().Last().Clone("sumMC")
ratio[v] = sumMC[v].Clone("ratio")
ratio[v].Clear()
ratio[v].Divide(sumMC[v],data[v][datak])

# Draw stack with MC contributions
stack[v].GetXaxis().SetLabelSize(0.04)
stack[v].GetXaxis().SetTitleSize(0.045)
stack[v].GetXaxis().SetTitleOffset(1.3)
stack[v].GetXaxis().SetTitle("m_{T}^{W#rightarrow l#nu} [GeV]")
stack[v].GetYaxis().SetTitle("Events")
stack[v].GetYaxis().SetLabelSize(0.04)
stack[v].GetYaxis().SetTitleSize(0.045)
stack[v].SetMaximum(1e7 * lumi/1000.)
stack[v].SetMinimum(1)

# Draw data
data[v][datak].SetMarkerStyle(20)
data[v][datak].SetMarkerSize(1.2)
data[v][datak].SetLineWidth(2)
data[v][datak].SetLineColor(R.kBlack)
data[v][datak].Draw("E SAME")

# Draw legend
legend[v].Draw("SAME")

# Add ATLAS label
text = R.TLatex()
text.SetNDC()
text.SetTextFont(72)
text.SetTextSize(0.045)
text.DrawLatex(0.21, 0.86, "ATLAS")
text.SetTextFont(42)
text.DrawLatex(0.21 + 0.09, 0.86, "Preliminary")
text.SetTextSize(0.04)
text.DrawLatex(0.21, 0.80, "#sqrt{{s}} = 13 TeV, {:.1f} fb^{{-1}}".format(lumi / 1000.0))

pad2.cd()
pad2.SetGridy()
pad2.SetTopMargin(0.01)
pad2.SetTickx(False)
pad2.SetTicky(False)
ratio[v].SetTitle("")
ratio[v].GetXaxis().SetLabelSize(0.15)
ratio[v].GetYaxis().SetLabelSize(0.15)
ratio[v].SetMaximum(2)
ratio[v].SetMinimum(2)
ratio[v].Draw("ep")

CPU times: user 110 ms, sys: 5.24 ms, total: 115 ms
Wall time: 115 ms


In [17]:
c1 = R.TCanvas("c1", "c1", 900, 700)
c1.Draw()
histo[v]["FNP"].GetValue().Draw("hist")